In [7]:
import pandas as pd
import os
import numpy as np
import re

ROOT = "/home/ubuntu/sla-operator" # change to match your workdir
CURRENT = os.path.join(ROOT, "pkg/data")
fp = os.path.join(CURRENT, "borg_traces_data.csv.zip")

In [3]:
df = pd.read_csv(os.path.join(CURRENT, "borg_traces_data.csv.zip"))
df.head()

,Unnamed: 0,time,instance_events_type,collection_id,scheduling_class,collection_type,priority,alloc_collection_id,instance_index,machine_id,...,assigned_memory,page_cache_memory,cycles_per_instruction,memory_accesses_per_instruction,sample_rate,cpu_usage_distribution,tail_cpu_usage_distribution,cluster,event,failed
0,0,0,2,94591244395,3,1,200,0,144,168846390496,...,0.014435,0.000415,NaN,NaN,1.0,[0.00314331 0.00381088 0.00401306 0.00415039 0...,[0.00535583 0.00541687 0.00548553 0.00554657 0...,7,FAIL,1
1,1,2517305308183,2,260697606809,2,0,360,221495397286,335,85515092,...,0.000000,0.000000,NaN,NaN,1.0,[1.23977661e-05 1.23977661e-05 1.23977661e-05 ...,[1.23977661e-05 1.23977661e-05 1.23977661e-05 ...,7,FAIL,1
2,2,195684022913,6,276227177776,2,0,103,0,376,169321752432,...,0.010422,0.000235,0.939919,0.001318,1.0,[0.01344299 0.01809692 0.0201416 0.02246094 0...,[0.02902222 0.02929688 0.0295105 0.0296936 0...,7,SCHEDULE,0
3,3,0,2,10507389885,3,0,200,0,1977,178294817221,...,0.041626,0.000225,1.359102,0.007643,1.0,[0.03704834 0.04125977 0.04290771 0.04425049 0...,[0.05535889 0.05584717 0.05633545 0.05718994 0...,8,FAIL,1
4,4,1810627494172,3,25911621841,2,0,0,0,3907,231364893292,...,0.000272,0.000010,NaN,NaN,1.0,[0. 0. 0. 0. 0...,[0.00041485 0.00041485 0.00041485 0.00041485 0...,2,FINISH,0


In [4]:
dummy = df[['resource_request', 'start_time', 'end_time', 'average_usage', 'maximum_usage', 'assigned_memory', 'event']].copy()
dummy.head()

,resource_request,start_time,end_time,average_usage,maximum_usage,assigned_memory,event
0,"{'cpus': 0.020660400390625, 'memory': 0.014434...",274800000000,275100000000,"{'cpus': 0.00466156005859375, 'memory': 0.0059...","{'cpus': 0.01190185546875, 'memory': 0.0059356...",0.014435,FAIL
1,"{'cpus': 0.00724029541015625, 'memory': 0.0013...",1800713000000,1800714000000,"{'cpus': 0.0, 'memory': 9.5367431640625e-07}","{'cpus': 0.0, 'memory': 9.5367431640625e-07}",0.000000,FAIL
2,"{'cpus': 0.048583984375, 'memory': 0.004165649...",81300000000,81600000000,"{'cpus': 0.024200439453125, 'memory': 0.002788...","{'cpus': 0.06005859375, 'memory': 0.0028457641...",0.010422,SCHEDULE
3,"{'cpus': 0.0704345703125, 'memory': 0.04162597...",1075500000000,1075800000000,"{'cpus': 0.047607421875, 'memory': 0.034423828...","{'cpus': 0.13330078125, 'memory': 0.03466796875}",0.041626,FAIL
4,"{'cpus': 0.00244903564453125, 'memory': 0.0002...",1565315000000,1565317000000,"{'cpus': 0.000270843505859375, 'memory': 7.629...","{'cpus': 0.00041484832763671875, 'memory': 7.6...",0.000272,FINISH


In [5]:
def split_cell(data):
    number = re.compile(r"[0-9]")
    if type(data) != float:
        data = data.split(": ")
        cpu = data[1]
        cpu = cpu.split(",")[0]
        cpu = float(cpu)
    
        memory = data[2].split("}")
        if number.match(memory[0]):
            memory = float(memory[0])
        else:
            memory = np.nan
    else:
        cpu, memory = np.nan, np.nan
    
    return [cpu, memory]

In [8]:
dummy['resource_request'].apply(split_cell)

0                [0.020660400390625, 0.014434814453125]
1          [0.00724029541015625, 0.0013027191162109375]
2                  [0.048583984375, 0.0041656494140625]
3                    [0.0704345703125, 0.0416259765625]
4         [0.00244903564453125, 0.00023174285888671875]
                              ...                      
405889           [0.03594970703125, 0.0154876708984375]
405890           [0.02197265625, 0.0013027191162109375]
405891               [0.1214599609375, 0.0826416015625]
405892     [0.00566864013671875, 0.0015621185302734375]
405893        [0.01629638671875, 0.0013027191162109375]
Name: resource_request, Length: 405894, dtype: object

In [20]:
dummy = dummy[dummy['event']=='FINISH']
dummy.shape

(92867, 7)

In [15]:
def remove_zeros(number):
    return int(re.sub(r'0', '', str(number)))

print(remove_zeros(2000030000))

23


In [21]:
data_df = pd.DataFrame()
data_df["cpu"] = dummy["resource_request"].apply(lambda x: split_cell(x)[0])
data_df["memory"] = dummy["resource_request"].apply(lambda x: split_cell(x)[1])
data_df["execution"] = abs(dummy["start_time"]-dummy["end_time"])
data_df["execution"] = data_df["execution"].apply(lambda x: remove_zeros(x))
data_df.reset_index(drop=True, inplace=True)
data_df.head()

,cpu,memory,execution
0,0.002449,0.000232,2
1,0.022003,0.001501,3
2,0.002518,0.000221,2
3,0.005669,0.002625,3
4,0.011154,0.001303,3


In [22]:
data_df["execution"].value_counts()

execution
3      50913
2      17780
241     6898
4       4133
1       2692
       ...  
46         1
156        1
272        1
236        1
234        1
Name: count, Length: 89, dtype: int64

In [ ]:
data_df.to_csv(os.path.join(CURRENT, "execution_time_data.csv"), index=0)